In [6]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

import pandas as pd
import re
import time
import numpy as np

In [2]:
email = input('Please enter your account(e-mail):')
password = input('Please enter your password:')

Please enter your account(e-mail):1
Please enter your password:1


In [79]:
# go into the Eruopresse website
url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
driver = webdriver.Chrome()
driver.get(url)
driver.find_element(By.ID, 'username').send_keys(email)
driver.find_element(By.ID, 'password').send_keys(password)
time.sleep(3)
driver.find_element(By.CLASS_NAME, 'btn-submit').click()

In [80]:
# choose the criteria of searching
driver.find_element(By.CLASS_NAME, 'lnk-text').click()
time.sleep(3)
driver.find_element(By.ID, 'specific-sources-rd').click()
time.sleep(5)
driver.find_element(By.ID, 'sf_247').click() # le monde

driver.find_element(By.XPATH, '//textarea').send_keys('climat') # key words : climat

In [81]:
# period: 1/oct/2022 - 1/dec/2022
Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value('2022')
Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value('2022')

Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value('10')
Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value('12')

Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value('1')
Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value('1')

In [82]:
driver.find_element(By.ID, 'btnSearch').click()

In [86]:
def scrolling():
    temp_height=0
 
    while True:
        
        driver.execute_script("window.scrollBy(0,1000)")
    
        check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
    
        if check_height==temp_height:
            break
            
        temp_height=check_height

In [87]:
for i in range(10):
    
    scrolling() 
    time.sleep(3)
    print('scrolling ', i+1, ' times' )

scrolling  1  times
scrolling  2  times
scrolling  3  times
scrolling  4  times
scrolling  5  times
scrolling  6  times
scrolling  7  times
scrolling  8  times
scrolling  9  times
scrolling  10  times


In [88]:
web_content = driver.page_source
obj = BeautifulSoup(web_content,'lxml').body
obj = obj.aside.ul

In [89]:
obj = obj.find_all(name='div', attrs={"class":"docListItem msDocItem"})

In [90]:
def get_information():
    
    info = pd.DataFrame(columns = ['title', 'date', 'publication', 'words', 'author-intro'])
    i = 0
    
    for obj_i in obj:
        
        obj_i = obj_i.div.next_sibling.next_sibling
        
        obj_i_1 = obj_i.div
        obj_i_1 = obj_i_1.find_all(name='span', attrs={"class":"source-name"})[0]
        obj_i_1 = str(obj_i_1)
        publication_i = re.findall(r"<span class=\"source-name\">(.+?)</span>", obj_i_1)[0]
        
        obj_i_2 = obj_i.div.next_sibling.next_sibling
        
        title_i = obj_i_2.div.a
        title_i = str(title_i)
        title_i = re.findall(r">(.+?)</a>", title_i)[0]
        
        obj_i_3 = obj_i_2.div.next_sibling.next_sibling 
        
        date_words_i = obj_i_3.div
        date_words_i = str(date_words_i)
        date_i = re.findall(r"<span class=\"details\">(.+?)<span class=", date_words_i)[0]
        date_i = date_i.replace(' ', '')
        
        words_i = re.findall(r"</span>(.+?)</span>", date_words_i)[0]
        words_i = words_i.replace(' ', '')
        
        author_intro_i = obj_i_3.div.next_sibling.next_sibling
        author_intro_i = str(author_intro_i)
        
        info.loc[i] = [title_i, date_i, publication_i, words_i, author_intro_i]
        i = i+1
        
    return info

In [165]:
info = get_information()

In [166]:
def clean_intro(x):
    try:
        x = x.split('<span class="doclist-author">')[1]
        author = x.split('</span>')[0]
        x = x.split('</span>')[1]
        
    except:
        x = x.split('</span> </span>')[1]
        author = None
    
    x = x.split('</div>')[0]
    x = re.sub(r"<([a-z]+)>",'',x)
    x = re.sub(r"</([a-z]+)>",'',x)
    x = re.sub('\u200a','',x)
    x = re.sub('-','',x)
    
    return author,x 

In [167]:
info['author-intro'] = info['author-intro'].apply(lambda x: clean_intro(x))

In [168]:
info['author'] = info['author-intro'].apply(lambda x: x[0])
info['intro'] = info['author-intro'].apply(lambda x: x[1])

In [169]:
info = info[['date', 'publication', 'words', 'author', 'title', 'intro']]

In [170]:
info.tail()

,date,publication,words,author,title,intro
373,2022-11-03,Le Monde,925mots,Richard Schittly,Meurtre par conjoint : débat complexe aux assises,"... par intermittence, sans leurs deux fi..."
374,2022-11-22,Le Monde,617mots,None,"La COP27, entre espoirs et désespoir",Le bilan des conférences mondiales sur...
375,2022-11-14,Le Monde,1221mots,propos recueillis parMathilde Gérard propos re...,Crise alimentaire : « le pire de la tempête » ...,... 16 novembre et de la COP27 ? Les diri...
376,2022-11-10,Le Monde,571mots,Roxana Azimi,Aurélie Filippetti va diriger les affaires cul...,"... de François Hollande, avait été salué..."
377,2022-11-19,Le Monde,1028mots,Rémi Dupré,Affaire Santini : des enregistrements contre l...,... le moindre comportement tel que décri...


In [174]:
import os

path = os.getcwd()
path = re.sub('Code','Result',path)

info.to_csv(path + '\\sample.csv')